In [42]:
pip install requests

In [43]:
import requests
import csv
from datetime import datetime

In [44]:
ACCESS_TOKEN = 'IGQWROYzIycW1XbmFLUWtVdlJLT2xjRDRuU0Q3aWtXb3VNWUE1NmJaNG1ic2gwM0pYUzlqYWRsOEJadktwdUtOZAGhEcFY0dVgzTi0xREM4VEVNRVBNUTl6Q0FBcTNlanc3cVVabXFhMWZAJVVpBamlXZAzRkWEVHUTQZD'
INSTAGRAM_ACCOUNT_ID = '565694226403515'
KEYWORD = "#Prabowo"
MAX_COMMENTS = 5000

In [45]:
def get_media_ids(instagram_account_id, access_token):
    url = f'https://graph.facebook.com/v17.0/{instagram_account_id}/media'
    params = {'access_token': access_token}
    response = requests.get(url, params=params)
    media_ids = [media['id'] for media in response.json().get('data', [])]
    return media_ids

In [46]:
def get_comments(media_id, access_token):
    comments = []
    url = f'https://graph.facebook.com/v17.0/{media_id}/comments'
    params = {
        'access_token': access_token,
        'fields': 'username,text,created_time'
    }
    while url:
        response = requests.get(url, params=params)
        data = response.json()
        for comment in data.get('data', []):
            if KEYWORD.lower() in comment['text'].lower():
                comments.append({
                    'username': comment['username'],
                    'comment': comment['text'],
                    'timestamp': comment['created_time']
                })
            if len(comments) >= MAX_COMMENTS:
                return comments
        url = data.get('paging', {}).get('next', None)
    return comments

In [47]:
media_ids = get_media_ids(INSTAGRAM_ACCOUNT_ID, ACCESS_TOKEN)

In [48]:
all_comments = []
for media_id in media_ids:
    comments = get_comments(media_id, ACCESS_TOKEN)
    all_comments.extend(comments)
    if len(all_comments) >= MAX_COMMENTS:
        break

In [49]:
csv_filename = "instagram_comments.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['username', 'comment', 'timestamp']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for comment in all_comments[:MAX_COMMENTS]:
        writer.writerow({
            'username': comment['username'],
            'comment': comment['comment'],
            'timestamp': datetime.strptime(comment['timestamp'], '%Y-%m-%dT%H:%M:%S+0000').strftime('%Y-%m-%d %H:%M:%S')
        })

print(f"Data berhasil disimpan dalam {csv_filename}. Jumlah komentar: {len(all_comments)}")

Data berhasil disimpan dalam instagram_comments.csv. Jumlah komentar: 0
